In [ ]:
import torch
import torch.nn as nn
from transformers import MBart50Tokenizer, MBartForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

class MultiTaskModel(nn.Module):
    def __init__(self, model_name="facebook/mbart-large-50"):
        super(MultiTaskModel, self).__init__()
        self.model = MBartForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer = MBart50Tokenizer.from_pretrained(model_name, legacy=False)

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs

    def generate(self, input_text, task_prefix, max_length=200):
        task_input = f"{task_prefix}: {input_text}"
        inputs = self.tokenizer(task_input, return_tensors="pt", padding=True, truncation=True, max_length=200)
        
        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            use_cache=True
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

def load_model(model_path, model_name):
    model = MultiTaskModel(model_name=model_name)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    tokenizer = MBart50Tokenizer.from_pretrained(model_name)    
    return model, tokenizer

def inference(model, tokenizer, task_prefix, input_text, max_length=200):
    model.eval()
    with torch.no_grad():
        output = model.generate(input_text=input_text, task_prefix=task_prefix, max_length=max_length)    
    return output

In [ ]:
## Test with new data
model, tokenizer = load_model(model_path="model/our_multi_task_model.pth", model_name="facebook/mbart-large-50")
model

MultiTaskModel(
  (model): MBartForConditionalGeneration(
    (model): MBartModel(
      (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (encoder): MBartEncoder(
        (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
        (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x MBartEncoderLayer(
            (self_attn): MBartSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): ReLU()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2)

In [ ]:
tasks = ["translation", "summarization", "qa"]
test_examples = [
    "we can't help remembering the things she did.",
    "ဂရိနဲ့ ဥရောပကြား ဆွေးနွေးမှုတွေ မပြေလည်သေး ငွေထုတ်ယူ ခွင့်ကို တကြိမ်မှာ ယူရို ၈၀ သာ ခွင့်ပြုဖို့ ကန့်သတ် လိမ့်မယ်လို့ အေသင်မြို့မှာ ရှိတဲ့ ဘီဘီစီ သတင်းထောက် က ပြောပါတယ်။ လောလောဆယ်မှာ ဂရိ အစိုးရက၊ မြီရှင် နိုင်ငံတွေရဲ့ ချေးငွေ စည်းကမ်းချက် တွေကို လိုက်နာဖို့ သင့်မသင့် ပြည်လုံးကျွတ် ဆန္ဒ ခံယူပွဲ ကျင်းပဖို့ အတွက် ဆွေးနွေး နေပါတယ်။ အဲဒီ စည်းကမ်းချက် တွေဟာ သူတို့ အတွက် အရှက်ရ စေပြီး သည်းမခံ နိုင်လောက် တဲ့ စည်းကမ်းတွေ ဖြစ်တယ်လို့ ဂရိ ဝန်ကြီးချုပ် Alexis Tsipras ကပြောပါတယ်။ ဒီစည်းကမ်းချက် တွေကို လိုက်နာမယ် မလိုက်နာဘူး ဆုံးဖြတ်ဖို့ ပြည်လုံးကျွတ် ဆန္ဒခံယူပွဲကို လာမယ့် ဇူလိုင်လ ၅ ရက်၊ တနင်္ဂနွေနေ့မှာ ကျင်းပဖို့ အစိုးရရဲ့ အဆိုပြုချက်ကို အတည်ပြုဖို့ ပါလီမန် အရေး ပေါ် အစည်းဝေး ကျင်းပမှာ ဖြစ်တယ်လို့ သူက ဆိုပါတယ်။ မစ္စတာ Tsipras ဟာ အင်္ဂါနေ့မှာ သက်တမ်း ကုန်တော့မယ့် လက်ရှိ ချေးငွေ သဘော တူညီချက်ကို ယာယီ သက်တမ်း တိုးဖို့ ယူရို ငွေကြေးသုံး နိုင်ငံ တွေက ဘဏ္ဍာရေး ဝန်ကြီးတွေနဲ့ ဘရပ်ဆဲ မြို့မှာ မကြာခင် တွေ့ဆုံဖို့ ရှိပါတယ်။",
    "Chocolate agar ဆိုတာ ဘာလဲ ရှင်းပြပါ"
]

for task, example in zip(tasks, test_examples):
    output = inference(model, tokenizer, task, example, max_length=200)
    print(f"Input: {example}")
    print(f"Output: {output}\n")

Input: we can't help remembering the things she did.
Output: ကျွန်တော်တို့ သူမ လုပ်ခဲ့တဲ့ အရာကို သတိမရဘဲ မနေနိုင်ဘူး။

Input: ဂရိနဲ့ ဥရောပကြား ဆွေးနွေးမှုတွေ မပြေလည်သေး ငွေထုတ်ယူ ခွင့်ကို တကြိမ်မှာ ယူရို ၈၀ သာ ခွင့်ပြုဖို့ ကန့်သတ် လိမ့်မယ်လို့ အေသင်မြို့မှာ ရှိတဲ့ ဘီဘီစီ သတင်းထောက် က ပြောပါတယ်။ လောလောဆယ်မှာ ဂရိ အစိုးရက၊ မြီရှင် နိုင်ငံတွေရဲ့ ချေးငွေ စည်းကမ်းချက် တွေကို လိုက်နာဖို့ သင့်မသင့် ပြည်လုံးကျွတ် ဆန္ဒ ခံယူပွဲ ကျင်းပဖို့ အတွက် ဆွေးနွေး နေပါတယ်။ အဲဒီ စည်းကမ်းချက် တွေဟာ သူတို့ အတွက် အရှက်ရ စေပြီး သည်းမခံ နိုင်လောက် တဲ့ စည်းကမ်းတွေ ဖြစ်တယ်လို့ ဂရိ ဝန်ကြီးချုပ် Alexis Tsipras ကပြောပါတယ်။ ဒီစည်းကမ်းချက် တွေကို လိုက်နာမယ် မလိုက်နာဘူး ဆုံးဖြတ်ဖို့ ပြည်လုံးကျွတ် ဆန္ဒခံယူပွဲကို လာမယ့် ဇူလိုင်လ ၅ ရက်၊ တနင်္ဂနွေနေ့မှာ ကျင်းပဖို့ အစိုးရရဲ့ အဆိုပြုချက်ကို အတည်ပြုဖို့ ပါလီမန် အရေး ပေါ် အစည်းဝေး ကျင်းပမှာ ဖြစ်တယ်လို့ သူက ဆိုပါတယ်။ မစ္စတာ Tsipras ဟာ အင်္ဂါနေ့မှာ သက်တမ်း ကုန်တော့မယ့် လက်ရှိ ချေးငွေ သဘော တူညီချက်ကို ယာယီ သက်တမ်း တိုးဖို့ ယူရို ငွေကြေးသုံး နိုင်ငံ တွေက ဘဏ္ဍာရေး ဝန်ကြီးတွေနဲ့ ဘရပ်ဆ

In [15]:
task = "translation"
example = "Miss Universe has to be fluent in English."

print(f"Task: {task}\nInput: {example}")
output = model.generate(example, task_prefix=task)
print(f"Output: {output}\n")

Task: translation
Input: Miss Universe has to be fluent in English.
Output: မယ်စက်ြာဝဠာဟာ အင်္ဂလိပ်စာကို သွက်လက် ချောမွေ့စွာ ပြောရပါမယ်။



In [8]:
task = "summarization"
example = "ဂရိနဲ့ ဥရောပကြား ဆွေးနွေးမှုတွေ မပြေလည်သေး ငွေထုတ်ယူ ခွင့်ကို တကြိမ်မှာ ယူရို ၈၀ သာ ခွင့်ပြုဖို့ ကန့်သတ် လိမ့်မယ်လို့ အေသင်မြို့မှာ ရှိတဲ့ ဘီဘီစီ သတင်းထောက် က ပြောပါတယ်။ လောလောဆယ်မှာ ဂရိ အစိုးရက၊ မြီရှင် နိုင်ငံတွေရဲ့ ချေးငွေ စည်းကမ်းချက် တွေကို လိုက်နာဖို့ သင့်မသင့် ပြည်လုံးကျွတ် ဆန္ဒ ခံယူပွဲ ကျင်းပဖို့ အတွက် ဆွေးနွေး နေပါတယ်။ အဲဒီ စည်းကမ်းချက် တွေဟာ သူတို့ အတွက် အရှက်ရ စေပြီး သည်းမခံ နိုင်လောက် တဲ့ စည်းကမ်းတွေ ဖြစ်တယ်လို့ ဂရိ ဝန်ကြီးချုပ် Alexis Tsipras ကပြောပါတယ်။ ဒီစည်းကမ်းချက် တွေကို လိုက်နာမယ် မလိုက်နာဘူး ဆုံးဖြတ်ဖို့ ပြည်လုံးကျွတ် ဆန္ဒခံယူပွဲကို လာမယ့် ဇူလိုင်လ ၅ ရက်၊ တနင်္ဂနွေနေ့မှာ ကျင်းပဖို့ အစိုးရရဲ့ အဆိုပြုချက်ကို အတည်ပြုဖို့ ပါလီမန် အရေး ပေါ် အစည်းဝေး ကျင်းပမှာ ဖြစ်တယ်လို့ သူက ဆိုပါတယ်။ မစ္စတာ Tsipras ဟာ အင်္ဂါနေ့မှာ သက်တမ်း ကုန်တော့မယ့် လက်ရှိ ချေးငွေ သဘော တူညီချက်ကို ယာယီ သက်တမ်း တိုးဖို့ ယူရို ငွေကြေးသုံး နိုင်ငံ တွေက ဘဏ္ဍာရေး ဝန်ကြီးတွေနဲ့ ဘရပ်ဆဲ မြို့မှာ မကြာခင် တွေ့ဆုံဖို့ ရှိပါတယ်။"

print(f"Task: {task}\nInput: {example}")
output = model.generate(example, task_prefix=task)
print(f"Output: {output}\n")

Task: summarization
Input: ဂရိနဲ့ ဥရောပကြား ဆွေးနွေးမှုတွေ မပြေလည်သေး ငွေထုတ်ယူ ခွင့်ကို တကြိမ်မှာ ယူရို ၈၀ သာ ခွင့်ပြုဖို့ ကန့်သတ် လိမ့်မယ်လို့ အေသင်မြို့မှာ ရှိတဲ့ ဘီဘီစီ သတင်းထောက် က ပြောပါတယ်။ လောလောဆယ်မှာ ဂရိ အစိုးရက၊ မြီရှင် နိုင်ငံတွေရဲ့ ချေးငွေ စည်းကမ်းချက် တွေကို လိုက်နာဖို့ သင့်မသင့် ပြည်လုံးကျွတ် ဆန္ဒ ခံယူပွဲ ကျင်းပဖို့ အတွက် ဆွေးနွေး နေပါတယ်။ အဲဒီ စည်းကမ်းချက် တွေဟာ သူတို့ အတွက် အရှက်ရ စေပြီး သည်းမခံ နိုင်လောက် တဲ့ စည်းကမ်းတွေ ဖြစ်တယ်လို့ ဂရိ ဝန်ကြီးချုပ် Alexis Tsipras ကပြောပါတယ်။ ဒီစည်းကမ်းချက် တွေကို လိုက်နာမယ် မလိုက်နာဘူး ဆုံးဖြတ်ဖို့ ပြည်လုံးကျွတ် ဆန္ဒခံယူပွဲကို လာမယ့် ဇူလိုင်လ ၅ ရက်၊ တနင်္ဂနွေနေ့မှာ ကျင်းပဖို့ အစိုးရရဲ့ အဆိုပြုချက်ကို အတည်ပြုဖို့ ပါလီမန် အရေး ပေါ် အစည်းဝေး ကျင်းပမှာ ဖြစ်တယ်လို့ သူက ဆိုပါတယ်။ မစ္စတာ Tsipras ဟာ အင်္ဂါနေ့မှာ သက်တမ်း ကုန်တော့မယ့် လက်ရှိ ချေးငွေ သဘော တူညီချက်ကို ယာယီ သက်တမ်း တိုးဖို့ ယူရို ငွေကြေးသုံး နိုင်ငံ တွေက ဘဏ္ဍာရေး ဝန်ကြီးတွေနဲ့ ဘရပ်ဆဲ မြို့မှာ မကြာခင် တွေ့ဆုံဖို့ ရှိပါတယ်။
Output: ဂရိ အစိုးရက ဘဏ်ကနေ ငွေထုတ်ယူခွင့် ကို ကန့်သတ် လိမ်

In [17]:
task = "qa"
example = "Flagella တို့၏ လုပ်ဆောင်ချက်ကို ဖော်ပြပါ။"

print(f"Task: {task}\nInput: {example}")
output = model.generate(example, task_prefix=task)
print(f"Output: {output}\n")

Task: qa
Input: Flagella တို့၏ လုပ်ဆောင်ချက်ကို ဖော်ပြပါ။
Output: Flagella သည် ဘက်တီးရီးယားများ၏ ရွေ့လျားမှု အတွက် အသုံးပြုသော ဆဲလ် ပြင်ပသို့ ထွက်နေသည့် ကြိုးသဖွယ် အတက်များ ဖြစ်သည်။ ၎င်းတို့သည် ဘက်တီးရီးယားအား ရေ၊ အာဟာရ သို့မဟုတ် host ဆဲလ်များဆီသို့ ရွှေ့လျားစေနိုင်သည်။

